In [ ]:
%matplotlib inline

## Iris introduction course
# 2. Loading and Saving

**Learning Outcome**: by the end of this section, you will be able to use Iris to load datasets from disk as Iris cubes and save Iris cubes back to disk.

**Duration:** 30 minutes.

**Overview:**<br>
2.1 [Iris Load Functions](#iris_load_functions)<br>
2.2 [Saving Cubes](#saving)<br>
2.3 [Exercise](#exercise_2)<br>
2.4 [Summary of the Section](#summary)

In [ ]:
import iris

----

## 2.1 Iris Load Functions<a id='iris_load_functions'></a>

There are three main load functions in Iris: ``load``, ``load_cube`` and ``load_cubes``.

1. **load** is a general purpose loading function. Typically this is where all data analysis will start, before more loading is refined with the more controlled loading from the other two functions.
2. **load_cube** returns a single cube from the given source(s) and constraint. There will be exactly one cube, or an exception will be raised.
3. **load_cubes** returns a cubelist of cubes from the given sources(s) and constraint(s). There will be exactly one cube per constraint, or an exception will be raised.


Note: ``load_cube`` is a special case of ``load``. 

Let's compare the result of calling `load` and `load_cube`. We start by selecting the `air_temp.pp` file from Iris' sample data.

In [ ]:
fname = iris.sample_data_path('air_temp.pp')

If we give this filepath to `load`, we see that Iris returns a cubelist.

In [ ]:
c1 = iris.load(fname)
print(type(c1))

If instead we give this filepath to `iris.load_cube`, we see that Iris returns a cube.

In [ ]:
c2 = iris.load_cube(fname)
print(type(c2))

If we compare the first cube in the cubelist returned by calling `load` and the cube returned by calling `load_cube` we see that they are the equal.

In [ ]:
c1[0] == c2

<div class="alert alert-block alert-warning">
<b><font color='brown'>Exercise: </font></b>Try loading in the `uk_hires.pp` from Iris' sample data, first with iris.load then iris.load_cube. Why does iris.load_cube fail if you only supply the filepath?
</div>

In [ ]:
#
# edit space for user code ...
#

----

## 2.2 Saving Cubes<a id='saving'></a>

The [iris.save](https://scitools.org.uk/iris/docs/latest/iris/iris.html#iris.save) function provides a convenient interface to save Cube and CubeList instances.

Below we load the `uk_hires.pp` file from Iris' provided sample data which returns a cubelist of the cubes that were produced from that file. We then save this cubelist out to netcdf.

In [ ]:
fname = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(fname)
iris.save(cubes, 'saved_cubes.nc')

We can check the ncdump to see what Iris saved:

In [ ]:
!ncdump -h saved_cubes.nc | head -n 20
!rm saved_cubes.nc

Extra keywords can be passed to specific fileformat savers.

<div class="alert alert-block alert-warning">
<b><font color='brown'>Exercise: </font></b>Take a look at the above link to the `iris.save` documentation to see what fileformats iris supports saving to.
</div>

----

## 2.3 Section Review Exercise<a id='exercise_2'></a>

1\. Load the file in `iris.sample_data_path('atlantic_profiles.nc')`, as in Exercise 1, but this time use `iris.load_cube` to load in the `sea_water_potential_temperature` cube only.

In [ ]:
# space for user code ...

2\. Check whether this cube has lazy data? Print the minimum, maximum, mean and standard deviation of the cube's data. Does the cube still have lazy data?

In [ ]:
# space for user code ...

3\. Go to the Iris reference documentation for ``iris.save``. What keywords are accepted to ``iris.save`` when saving a PP file?

In [ ]:
# space for user code ...

#### Exercise Solutions: 
Once you are happy with your answers, you can check the solutions by runnning the cell below:

In [ ]:
# SAMPLE SOLUTION
# %load solutions/iris_exercise_2

-----

## 2.4 Section Summary : Loading and Saving<a id='summary'></a>

In this section we learnt:
* Iris has different loading functions for different purposes, for example `iris.load` should be used for exploratory data analysis
* Iris supports saving to three fileformats: netCDF, GRIB2 and PP